In [1]:
import geopandas as gpd

In [2]:
shapefile = gpd.read_file("delhi_ncr_region.geojson")


In [3]:
shapefile_utm = shapefile.to_crs(epsg=32644)

In [4]:
minx, miny, maxx, maxy = shapefile_utm.total_bounds

In [5]:
print("Bounding Box Coordinates (in meters):")
print(f"minx = {minx}, miny = {miny}, maxx = {maxx}, maxy = {maxy}")

Bounding Box Coordinates (in meters):
minx = -39135.081494243466, miny = 2958988.235697051, maxx = 252193.69892669076, maxy = 3324410.040147016


In [6]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import box, Point


In [7]:
minx, miny, maxx, maxy =  -39135.081494243466, 2958988.235697051, 252193.69892669076, 3324410.040147016
grid_size = 60000  # 60 km


In [8]:
grid_cells = []
centers = []
corner_points = []

In [13]:
for x0 in np.arange(minx, maxx, grid_size):
    for y0 in np.arange(miny, maxy, grid_size):
        x1 = x0 + grid_size
        y1 = y0 + grid_size
        cell = box(x0, y0, x1, y1)
        grid_cells.append(cell)
        cx, cy = (x0 + x1) / 2, (y0 + y1) / 2
        centers.append(Point(cx, cy))

In [14]:
corner_points.extend([
            Point(x0, y0),  # bottom-left
            Point(x0, y1),  # top-left
            Point(x1, y0),  # bottom-right
            Point(x1, y1)   # top-right
        ])


In [15]:
grid_shapefile = gpd.GeoDataFrame(geometry=grid_cells, crs="EPSG:32644")
centers_shapefile  = gpd.GeoDataFrame(geometry=centers, crs="EPSG:32644")
corners_shapefile  = gpd.GeoDataFrame(geometry=corner_points, crs="EPSG:32644")

In [16]:
grid_wgs = grid_shapefile.to_crs(epsg=4326)
centers_wgs = centers_shapefile.to_crs(epsg=4326)
corners_wgs = corners_shapefile.to_crs(epsg=4326)


In [17]:
center_coords = pd.DataFrame({
    "lat": centers_wgs.geometry.y,
    "lon": centers_wgs.geometry.x
})
print("Grid Cell Centers (Lat/Lon):")
print(center_coords.head())

Grid Cell Centers (Lat/Lon):
         lat        lon
0  26.929788  75.875616
1  27.469304  75.850822
2  28.008756  75.825328
3  28.548145  75.799119
4  29.087467  75.772179


In [18]:
image_centers = [
    {"id": "img1", "lat": 26.929788, "lon": 75.875616},
    {"id": "img2", "lat": 27.469304, "lon": 75.850822},
    {"id": "img3", "lat": 28.008756, "lon": 75.825328},
    {"id": "img4", "lat": 28.548145, "lon": 75.799119},
    {"id": "img5", "lat": 29.087467, "lon": 75.772179},
]

In [19]:
df = pd.DataFrame(image_centers)
df["geometry"] = [Point(lon, lat) for lat, lon in zip(df["lat"], df["lon"])]
image_gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")


In [20]:
filtered = gpd.sjoin(image_gdf, grid_wgs, predicate="within", how="inner")

In [21]:
print("\nImages Whose Centers Fall Within Grid:")
print(filtered[["id", "lat", "lon"]])


Images Whose Centers Fall Within Grid:
     id        lat        lon
0  img1  26.929788  75.875616
0  img1  26.929788  75.875616
1  img2  27.469304  75.850822
1  img2  27.469304  75.850822
2  img3  28.008756  75.825328
2  img3  28.008756  75.825328
3  img4  28.548145  75.799119
3  img4  28.548145  75.799119
4  img5  29.087467  75.772179
4  img5  29.087467  75.772179


In [22]:
num_before = image_gdf.shape[0]

In [23]:
num_after = filtered.shape[0]

In [24]:
print(f"\nNumber of images BEFORE filtering: {num_before}")
print(f"Number of images AFTER filtering:  {num_after}")


Number of images BEFORE filtering: 5
Number of images AFTER filtering:  10
